In [ ]:
import vtk
import vmtk
from vmtk import vmtkscripts
from vmtk import vmtkcenterlines

def find_equidistant_points(input_mesh_file, output_centerline_file, target_distance):
    """
    Finds equidistant points along the centerline of a tubular mesh using a more robust VMTK workflow.
    """

    # 1. Read the mesh
    if input_mesh_file.lower().endswith(".vtp"):
        reader = vtk.vtkXMLPolyDataReader()
    elif input_mesh_file.lower().endswith(".ply"):
        reader = vtk.vtkPLYReader()
    else:
        raise ValueError("Unsupported file format. Only .vtp and .ply files are supported.")
    
    reader.SetFileName(input_mesh_file)
    reader.Update()
    mesh = reader.GetOutput()

    # 2. Extract centerline using a more robust workflow
    centerlines = vmtkcenterlines.vmtkCenterlines()
    centerlines.Surface = mesh
    centerlines.SeedIds = [0]  # Specify a seed point (adjust as needed)
    centerlines.Execute()

    # Get the output centerline
    centerline = centerlines.Centerlines

    # 3. Resample centerline
    resampler = vmtkscripts.vmtkCenterlineResampling()
    resampler.Centerlines = centerline
    resampler.TargetDistance = target_distance
    resampler.Execute()
    resampled_centerline = resampler.Centerlines

    # 4. Write Resampled Centerline
    writer = vtk.vtkXMLPolyDataWriter()
    writer.SetFileName(output_centerline_file)
    writer.SetInputData(resampled_centerline)
    writer.Write()

    # 5. Visualize (Optional)
    mesh_mapper = vtk.vtkPolyDataMapper()
    mesh_mapper.SetInputData(mesh)
    mesh_actor = vtk.vtkActor()
    mesh_actor.SetMapper(mesh_mapper)
    mesh_actor.GetProperty().SetOpacity(0.5)

    resampled_mapper = vtk.vtkPolyDataMapper()
    resampled_mapper.SetInputData(resampled_centerline)
    resampled_actor = vtk.vtkActor()
    resampled_actor.SetMapper(resampled_mapper)
    resampled_actor.GetProperty().SetColor(1, 0, 0)

    renderer = vtk.vtkRenderer()
    render_window = vtk.vtkRenderWindow()
    render_window.AddRenderer(renderer)

    interactor = vtk.vtkRenderWindowInteractor()
    interactor.SetRenderWindow(render_window)

    renderer.AddActor(mesh_actor)
    renderer.AddActor(resampled_actor)
    renderer.SetBackground(0.1, 0.2, 0.4)

    render_window.Render()
    interactor.Start()

if __name__ == "__main__":
    input_mesh_file = r"C:\Users\Lenovo\OneDrive - Syddansk Universitet\Dokumenter\GitHub\Broncho-Project\code\pointclouds\meshes\accumulated_point_cloud.ply"  # Change to .ply file if needed
    output_centerline_file = "resampled_centerline.vtp"
    target_distance = 1.0
    find_equidistant_points(input_mesh_file, output_centerline_file, target_distance)

Cleaning surface.
Triangulating surface.
Capping surface.
Please position the mouse and press space to add source points, 'u' to undo
